

---


<h3><center>MScFE 690 Capstone</center></h3>
<h3><center>Group 14</center></h3>
<h3><center>Predicting Indian Stock Market Index Movement using Macroeconomic Data and Machine Learning</center></h3>


---

<h3><center>Step 1: Capturing Input Data</center></h3>


Following Macroeconomic variables and Indexs are captured and analyzed for this paper are:

*   FII/DII Activity
*   GDP Growth rate
*   Crude Oil
*   Monetary Policy
*   USD/INR Exchange rates
*   US Global and Indian Market Index
*   Industrial Production
*   CPI

In this notebook, we will download, clean and consolidate these macroeconomic and index data for further steps.

In [ ]:
!pip install yfinance==0.1.74
!pip install pandas_datareader==0.10.0
!pip install nsepython

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import yfinance as yf
yf.pdr_override()
from pandas_datareader import data as pdr
import pandas as pd
import nsepython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Accumulate data for last 25 years
start = datetime.datetime (1996, 4, 30)
end = datetime.datetime (2022, 5, 1)
idx = pd.date_range(start, end)

# Picking NSE Data from the NSE python library as Yahoo finance do not have entire history of NSE data.
nse_data = nsepython.index_history("NIFTY 50", start.strftime('%d-%b-%Y'), end.strftime('%d-%b-%Y'))
nse_data['HistoricalDate'] = pd.to_datetime(nse_data['HistoricalDate'])
nse_data.drop(['Index Name', 'INDEX_NAME', 'OPEN', 'HIGH', 'LOW'], axis=1, inplace=True)
nse_data.rename(columns={'CLOSE': 'NSE Adj Close'}, inplace=True)
nse_data = nse_data.set_index('HistoricalDate')
nse_data = nse_data.reindex(idx)
# Ffill for holidays
nse_data = nse_data.fillna(method='ffill')
nse_data.index = nse_data.index.strftime('%Y-%m-%d')
nse_data.index.names = ['Date']

# Picking Dow Jones data from yahoo finance
tickers = ['^DJI'] # Dow Jones index
dow_data = pdr.get_data_yahoo(tickers, start=start, end=end)['Adj Close'].to_frame()
dow_data = dow_data.reindex(idx)
# Ffill for holidays
dow_data = dow_data.fillna(method='ffill')
dow_data.index = dow_data.index.strftime('%Y-%m-%d')
dow_data.rename(columns={'Adj Close': 'Dow Jones Adj Close'}, inplace=True)

[*********************100%***********************]  1 of 1 completed


In [ ]:
# Merge the indexs data in single data frames
indexs_data = pd.concat([nse_data, dow_data], axis = 1)
indexs_data

,NSE Adj Close,Dow Jones Adj Close
1996-04-30,1114.30,5569.080078
1996-05-01,1137.80,5575.220215
1996-05-02,1132.00,5498.270020
1996-05-03,1119.40,5478.029785
1996-05-04,1119.40,5478.029785
...,...,...
2022-04-27,17038.40,33301.929688
2022-04-28,17245.05,33916.390625
2022-04-29,17102.55,32977.210938
2022-04-30,17102.55,32977.210938


In [ ]:
# FII/DII Activity for NSE Index (https://www.moneycontrol.com/stocks/marketstats/fii_dii_activity/index.php)
fii_dii_activity = pd.read_csv("./FII DII Activity.csv")
fii_dii_all_activity = fii_dii_activity.set_index('Date')
fii_dii_net_activity = fii_dii_all_activity[['FII Net Purchase or Sales', 'DII Net Purchase or Sales']]

# Normalized GDP Data for India (https://fred.stlouisfed.org/series/INDLORSGPNOSTSAM)
gdp_india = pdr.get_data_fred('INDLORSGPNOSTSAM',start, end)
gdp_india.index = gdp_india.index.strftime('%Y-%m-%d')
gdp_india.rename(columns={'INDLORSGPNOSTSAM': 'Norm GDP'}, inplace=True)

# Capturing the oil data
oil_data = pdr.get_data_fred('POILBREUSDM', start, end)
oil_data.index = oil_data.index.strftime('%Y-%m-%d')
oil_data.rename(columns={'POILBREUSDM': 'Brent Crude Price'}, inplace=True)

# CPI Monthly - INDCPIALLMINMEI
cpi_monthly = pdr.get_data_fred('INDCPIALLMINMEI',start, end)
cpi_monthly.index = cpi_monthly.index.strftime('%Y-%m-%d')
cpi_monthly.rename(columns={'INDCPIALLMINMEI': 'CPI'}, inplace=True)

# Industrial production - INDPROINDMISMEI
iip_monthly = pdr.get_data_fred('INDPROINDMISMEI',start, end)
iip_monthly.index = iip_monthly.index.strftime('%Y-%m-%d')
iip_monthly.rename(columns={'INDPROINDMISMEI': 'IIP'}, inplace=True)

# Interest rates - INDIR3TIB01STM monthly or IRSTCB01INM156N Immediate monthly
short_rates_india = pdr.get_data_fred('IRSTCB01INM156N',start, end)
short_rates_india.index = short_rates_india.index.strftime('%Y-%m-%d')
short_rates_india.rename(columns={'IRSTCB01INM156N': 'Central Bank Rates'}, inplace=True)

# Indian Rupees to U.S. Dollar Spot Exchange Rate (DEXINUS)
usd_inr_rates = pdr.get_data_fred('DEXINUS', start, end)
# To accomodate for missing dates in fred data, use ffill for all missing dates
usd_inr_rates = usd_inr_rates.reindex(idx)
usd_inr_rates.index = usd_inr_rates.index.strftime('%Y-%m-%d')
usd_inr_rates.rename(columns={'DEXINUS': 'USD INR Rate'}, inplace=True)
usd_inr_rates = usd_inr_rates.fillna(method='ffill')

# Merge the macro economic variables data in single data frames
macro_data = pd.concat([gdp_india, oil_data, cpi_monthly, iip_monthly, short_rates_india, usd_inr_rates, fii_dii_net_activity], axis = 1)#.dropna()
macro_data

,Norm GDP,Brent Crude Price,CPI,IIP,Central Bank Rates,USD INR Rate,FII Net Purchase or Sales,DII Net Purchase or Sales
1996-05-01,100.928058,19.319130,27.196636,34.956089,12.0,34.80,NaN,NaN
1996-06-01,100.835625,18.577500,27.611219,34.513567,12.0,35.23,NaN,NaN
1996-07-01,100.743785,19.714783,28.108718,34.114211,12.0,35.10,NaN,NaN
1996-08-01,100.653025,20.605909,28.440385,34.920563,12.0,35.65,NaN,NaN
1996-09-01,100.563376,23.081429,28.523301,34.590072,12.0,35.85,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-04-30,NaN,NaN,NaN,NaN,NaN,76.48,NaN,NaN
2022-09-01,NaN,NaN,NaN,NaN,NaN,NaN,22025.62,-7068.63
2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,-6567.71,10546.02
2022-07-01,NaN,NaN,NaN,NaN,NaN,NaN,-58112.37,46599.23


In [ ]:
# Merge index data with macro data
all_variables = pd.concat([indexs_data, macro_data], axis = 1)

# Unfortunately, we don't have macroeconomic variables on a daily basis. Hence,
# Filter the data frame based on the available data
all_variables = all_variables[~pd.isnull(all_variables['Norm GDP']) & 
                              ~pd.isnull(all_variables['Brent Crude Price']) &
                              ~pd.isnull(all_variables['CPI']) & 
                              ~pd.isnull(all_variables['IIP']) & 
                              ~pd.isnull(all_variables['Central Bank Rates'])]

all_variables.index.name = 'Date'
all_variables

,NSE Adj Close,Dow Jones Adj Close,Norm GDP,Brent Crude Price,CPI,IIP,Central Bank Rates,USD INR Rate,FII Net Purchase or Sales,DII Net Purchase or Sales
Date,,,,,,,,,,
1996-05-01,1137.80,5575.220215,100.928058,19.319130,27.196636,34.956089,12.00,34.80,NaN,NaN
1996-06-01,1089.90,5643.180176,100.835625,18.577500,27.611219,34.513567,12.00,35.23,NaN,NaN
1996-07-01,1109.00,5729.979980,100.743785,19.714783,28.108718,34.114211,12.00,35.10,NaN,NaN
1996-08-01,1036.60,5594.750000,100.653025,20.605909,28.440385,34.920563,12.00,35.65,NaN,NaN
1996-09-01,1029.00,5616.209961,100.563376,23.081429,28.523301,34.590072,12.00,35.85,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-01-01,17354.05,36338.300781,99.574644,85.622381,137.830179,119.009911,4.25,74.39,-35493.59,31231.05
2022-02-01,17576.85,35405.238281,99.517528,94.267000,137.720003,117.988724,4.25,74.78,-41346.35,21928.40
2022-03-01,16793.90,33294.949219,99.423298,112.440000,138.821763,119.245588,4.25,75.79,-45720.07,42084.07


In [ ]:
# Save all data to a csv for next steps (feature engineering and ML)
all_variables.to_csv('indexs_and_macroeconomic_data.csv')